In [1]:
import re
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from torch.nn.utils.rnn import pad_sequence
from tqdm.auto import tqdm
from datasets import Dataset, DatasetDict, load_dataset
from typing import Dict, List, Tuple, Callable, Any
from configuration import CFG

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/Users/qcqced/Desktop/SAMSUNG/venv/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [42]:
""" Helper Function """

def select_alphanumeric_and_non_english(text: str) -> str:
    pattern = re.compile(r'[^\w\d\s]|_')
    result = pattern.sub('', text)
    return result

def select_tokens(tokens: List[str]) -> List[str]:
    selected_tokens = [token for token in tokens if re.match(r'^[\w\d\s]+$', token) and '_' not in token]
    return selected_tokens

def select_string(token: str) -> str:
    flag = True if re.match(r'^[\w\d\s]+$', token) else False
    return selected_token

text = "Hello, World! 123 / Example_____ 테스트 文字列"
result = select_alphanumeric_and_non_english(text)
print(result)

Hello World 123  Example 테스트 文字列


In [45]:
import re

def is_valid_start(token, pattern):
    # 정규표현식: 알파벳, 숫자, 다른 언어의 문자열 선택 (언더스코어 제외)
    regex_pattern = re.compile(r'[^\w\d\s]|_')
    
    # 주어진 문자열이 특정 패턴으로 시작하며, 언더스코어를 포함하지 않는지 확인
    return token.startswith(pattern) and not bool(regex_pattern.search(token))

# 예시 토큰 리스트
tokens = ['▁trained', ',', '▁train', ',', '▁Pre', 'trained', ',', '▁Pre', 'training', '.', '▁Pre', 'trained', ',', '▁Pre', 'training', ',', '▁Pre', 'trained', ',', '▁Pre', 'training', ',', '▁Pre', 'trained', ',', '▁Pre', 'training', ',', '▁Pre', 'trained', ',', '▁Pre', 'training', ',', '▁Pre', 'trained', ',', '▁Pre', 'training', ',', '▁Pre', 'trained', ',', '▁Pre', 'training', ',', '▁Pre', 'trained', ',', '▁Pre', 'training', ',', '▁Pre', 'trained', ',', '▁Pre', 'training', ',', '▁Pre', 'trained', ',', '▁Pre', 'training', ',', '▁Pre']

# 특정 패턴으로 시작하는 토큰만 선택
pattern_to_start_with = '▁Pre'
selected_tokens = [token for token in tokens if is_valid_start(token, pattern_to_start_with)]

print(selected_tokens)


[]


In [37]:
from transformers import AutoTokenizer
import torch
import random

def whole_word_masking(text, tokenizer, mlm_probability=0.15):
    # Tokenize the text using the Unicode tokenizer
    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(text)))
    print(select_alphanumeric_and_non_english(str(tokens)))
    # Whole Word Masking
    mask_labels = [0] * len(tokens)
    print(mask_labels)
    for i, token in enumerate(tokens):
        # Set probability of masking each word
        if random.random() < mlm_probability:
            mask_labels[i] = 1

            # Find the boundaries of the whole word
            j = i + 1
            while j < len(tokens) and tokens[j].startswith(""):
                mask_labels[j] = 1
                j += 1

    # Convert mask labels to tensor
    mask_labels = torch.tensor(mask_labels, dtype=torch.long)

    # Apply masking to input tokens
    input_ids = tokenizer.encode(text, add_special_tokens=True)
    input_ids = torch.tensor(input_ids, dtype=torch.long)

    input_ids[mask_labels == 1] = tokenizer.mask_token_id

    return input_ids, mask_labels

# Example usage:
# Replace 'your_unicode_tokenizer' with the actual model name or path
tokenizer = CFG.tokenizer
# Sample input for testing
input_text = "trained, train, Pretrained, Pretraining. Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining. Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining"

# Apply Whole Word Masking
masked_input, mask_labels = whole_word_masking(input_text, tokenizer)

# Print results
print("Masked Text:", tokenizer.decode(masked_input.tolist()))
print("Mask Labels:", mask_labels.tolist())


CLS trained  train  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training  Pre trained  Pre training SEP
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [64]:
from transformers import AutoTokenizer, DataCollatorForWholeWordMask
import torch

# Example input text
input_text = "This is an example sentence for Span Masking algorithm. It is important to keep the token length below 100 for demonstration purposes."

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-large')

# Tokenize input text
input_tokens = tokenizer.tokenize(input_text)

# Whole Word Masking: Create a list of random labels (1 for [MASK], 0 for others)
mask_labels = [1 if token.startswith("▁") else 0 for token in input_tokens]

# Whole Word Masking: Apply masking to input tokens
input_tokens = tokenizer.convert_tokens_to_ids(input_tokens)
masked_tokens = DataCollatorForWholeWordMask(tokenizer)([input_tokens], mask_labels=mask_labels)

# Convert masked tokens back to text for visualization
masked_text = tokenizer.decode(masked_tokens[0], skip_special_tokens=True)

# Print results
print("Input Text:", input_text)
print("Masked Text:", masked_text)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/Users/qcqced/Desktop/SAMSUNG/venv/lib/python3.9/site-packages/transformers/data/data_collator.py:951: UserWarning: DataCollatorForWholeWordMask is only suitable for BertTokenizer-like tokenizers. Please refer to the documentation for more information.
  warnings.warn(


KeyError: 0

In [115]:
mlm_probability = 0.30
input_tokens = """
trained, train, Pretrained, Pretraining. Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining. Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, Pretrained, Pretraining, 
"""
# tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
tokenizer = CFG.tokenizer
input_tokens = tokenizer(
    input_tokens,
    return_offsets_mapping=True,
)
input_tokens["input_ids"]


def get_padding_mask(input_id: Tensor) -> Tensor:
    return torch.zeros(input_id.shape).bool()


def _whole_word_mask(
        input_tokens: List[str],
        max_predictions: int = CFG.max_seq
) -> List[int]:
    cand_indexes = []
    for i, token in enumerate(input_tokens):
        if token == "[CLS]" or token == "[SEP]":
            continue

        if len(cand_indexes) >= 1 and token.startswith("▁"):
            cand_indexes[-1].append(i)
        else:
            cand_indexes.append([i])

    random.shuffle(cand_indexes)
    num_to_predict = min(max_predictions, max(1, int(round(len(input_tokens) * mlm_probability))))
    masked_lms = []
    covered_indexes = set()
    for index_set in cand_indexes:
        if len(masked_lms) >= num_to_predict:
            break
        if len(masked_lms) + len(index_set) > num_to_predict:
            continue
        is_any_index_covered = False
        for index in index_set:
            if index in covered_indexes:
                is_any_index_covered = True
                break
        if is_any_index_covered:
            continue
        for index in index_set:
            covered_indexes.add(index)
            masked_lms.append(index)

    if len(covered_indexes) != len(masked_lms):
        raise ValueError("Length of covered_indexes is not equal to length of masked_lms.")
    mask_labels = [1 if i in covered_indexes else 0 for i in range(len(input_tokens))]
    return mask_labels

def get_mask_tokens(inputs, mask_labels):
    """ Prepare masked tokens inputs/labels for masked language modeling(15%):
    80% MASK, 10% random, 10% original. Set 'mask_labels' means we use whole word mask (wwm), we directly mask idxs according to it's ref
    """
    labels = inputs.clone()
    probability_matrix = mask_labels

    special_tokens_mask = [
        tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
    ]
    probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)
    if tokenizer.pad_token is not None:
        padding_mask = labels.eq(tokenizer.pad_token_id)
        probability_matrix.masked_fill_(padding_mask, value=0.0)

    masked_indices = probability_matrix.bool()
    labels[~masked_indices] = -100  # We only compute loss on masked tokens

    # 80% of the time, we replace masked input tokens with tokenizer.mask_token ([MASK])
    indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
    inputs[indices_replaced] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)

    # 10% of the time, we replace masked input tokens with random word
    indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
    random_words = torch.randint(len(tokenizer), labels.shape, dtype=torch.long)
    inputs[indices_random] = random_words[indices_random]

    # The rest of the time (10% of the time) we keep the masked input tokens unchanged
    return inputs, labels

def testing(batched):
    """ Masking for MLM with whole-word tokenizing """
    batched = batched["input_ids"]
    input_ids = [torch.tensor(batched)]
    padding_mask = [get_padding_mask(x) for x in input_ids]
    padding_mask = pad_sequence(padding_mask, batch_first=True, padding_value=True)
    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=0)

    mask_labels = []
    ref_tokens = []
    for input_id in batched:
        token = tokenizer._convert_id_to_token(input_id)
        ref_tokens.append(token)
    mask_labels.append(_whole_word_mask(ref_tokens))

    mask_labels = [torch.tensor(x) for x in mask_labels]
    mask_labels = pad_sequence(mask_labels, batch_first=True, padding_value=0)
    inputs, labels = get_mask_tokens(
        input_ids,
        mask_labels
    )
    return inputs, labels

inputs, labels = testing(input_tokens)
print(inputs, end="\n\n")
print(labels)

tensor([[     1,   3266,    261, 128000,    261,   3810,  16676,    261,   3810,
          18782,    260,   3810, 128000,    261,   3810, 128000,    261, 128000,
         128000, 128000, 128000, 128000, 128000, 128000,  16676,    261,   3810,
          18782,    261,   3810,  16676,    261,   3810,  18782,    261,   3810,
          16676, 128000, 102707,  18782,    261,   3810,  16676, 128000, 128000,
          18782,    261,   3810,  16676,    261,   3810,  18782, 128000, 128000,
          16676,    261,   3810,  18782,    261,   3810,  16676, 128000, 128000,
          18782,    261,   3810,  16676,    261,   3810,  18782,    261,   3810,
         128000,    261,   3810,  18782, 128000,   3810,  16676, 128000, 128000,
          18782,    261,   3810,  16676,    261,   3810,  18782,    261,   3810,
          16676, 128000,   3810,  18782,    261,   3810,  16676,    261,   3810,
          18782,    260,   3810,  16676,    261,   3810,  18782,    261,   3810,
         128000,    261,   3

In [114]:
tokenizer._convert_id_to_token(1010)

','

In [84]:
input_tokens.tokens() 

['[CLS]',
 '▁trained',
 ',',
 '▁train',
 ',',
 '▁Pre',
 'trained',
 ',',
 '▁Pre',
 'training',
 '.',
 '▁Pre',
 'trained',
 ',',
 '▁Pre',
 'training',
 ',',
 '▁Pre',
 'trained',
 ',',
 '▁Pre',
 'training',
 ',',
 '▁Pre',
 'trained',
 ',',
 '▁Pre',
 'training',
 ',',
 '▁Pre',
 'trained',
 ',',
 '▁Pre',
 'training',
 ',',
 '▁Pre',
 'trained',
 ',',
 '▁Pre',
 'training',
 ',',
 '▁Pre',
 'trained',
 ',',
 '▁Pre',
 'training',
 ',',
 '▁Pre',
 'trained',
 ',',
 '▁Pre',
 'training',
 ',',
 '▁Pre',
 'trained',
 ',',
 '▁Pre',
 'training',
 ',',
 '▁Pre',
 'trained',
 ',',
 '▁Pre',
 'training',
 ',',
 '▁Pre',
 'trained',
 ',',
 '▁Pre',
 'training',
 ',',
 '▁Pre',
 'trained',
 ',',
 '▁Pre',
 'training',
 ',',
 '▁Pre',
 'trained',
 ',',
 '▁Pre',
 'training',
 ',',
 '▁Pre',
 'trained',
 ',',
 '▁Pre',
 'training',
 ',',
 '▁Pre',
 'trained',
 ',',
 '▁Pre',
 'training',
 ',',
 '▁Pre',
 'trained',
 ',',
 '▁Pre',
 'training',
 '.',
 '▁Pre',
 'trained',
 ',',
 '▁Pre',
 'training',
 ',',
 '▁Pre',
 'trained'

In [85]:
input_tokens.word_ids()

[None,
 0,
 0,
 1,
 1,
 2,
 2,
 2,
 3,
 3,
 3,
 4,
 4,
 4,
 5,
 5,
 5,
 6,
 6,
 6,
 7,
 7,
 7,
 8,
 8,
 8,
 9,
 9,
 9,
 10,
 10,
 10,
 11,
 11,
 11,
 12,
 12,
 12,
 13,
 13,
 13,
 14,
 14,
 14,
 15,
 15,
 15,
 16,
 16,
 16,
 17,
 17,
 17,
 18,
 18,
 18,
 19,
 19,
 19,
 20,
 20,
 20,
 21,
 21,
 21,
 22,
 22,
 22,
 23,
 23,
 23,
 24,
 24,
 24,
 25,
 25,
 25,
 26,
 26,
 26,
 27,
 27,
 27,
 28,
 28,
 28,
 29,
 29,
 29,
 30,
 30,
 30,
 31,
 31,
 31,
 32,
 32,
 32,
 33,
 33,
 33,
 34,
 34,
 34,
 35,
 35,
 35,
 36,
 36,
 36,
 37,
 37,
 37,
 38,
 38,
 38,
 39,
 39,
 39,
 40,
 40,
 40,
 41,
 41,
 41,
 42,
 42,
 42,
 43,
 43,
 43,
 44,
 44,
 44,
 45,
 45,
 45,
 46,
 46,
 46,
 47,
 47,
 47,
 48,
 48,
 48,
 49,
 49,
 49,
 50,
 50,
 50,
 51,
 51,
 51,
 52,
 52,
 52,
 53,
 53,
 53,
 54,
 54,
 54,
 55,
 55,
 55,
 56,
 56,
 56,
 57,
 57,
 57,
 58,
 58,
 58,
 59,
 59,
 59,
 60,
 60,
 60,
 61,
 61,
 61,
 None]

In [88]:
input_tokens.offset_mapping

[(0, 0),
 (1, 8),
 (8, 9),
 (9, 15),
 (15, 16),
 (16, 20),
 (20, 27),
 (27, 28),
 (28, 32),
 (32, 40),
 (40, 41),
 (41, 45),
 (45, 52),
 (52, 53),
 (53, 57),
 (57, 65),
 (65, 66),
 (66, 70),
 (70, 77),
 (77, 78),
 (78, 82),
 (82, 90),
 (90, 91),
 (91, 95),
 (95, 102),
 (102, 103),
 (103, 107),
 (107, 115),
 (115, 116),
 (116, 120),
 (120, 127),
 (127, 128),
 (128, 132),
 (132, 140),
 (140, 141),
 (141, 145),
 (145, 152),
 (152, 153),
 (153, 157),
 (157, 165),
 (165, 166),
 (166, 170),
 (170, 177),
 (177, 178),
 (178, 182),
 (182, 190),
 (190, 191),
 (191, 195),
 (195, 202),
 (202, 203),
 (203, 207),
 (207, 215),
 (215, 216),
 (216, 220),
 (220, 227),
 (227, 228),
 (228, 232),
 (232, 240),
 (240, 241),
 (241, 245),
 (245, 252),
 (252, 253),
 (253, 257),
 (257, 265),
 (265, 266),
 (266, 270),
 (270, 277),
 (277, 278),
 (278, 282),
 (282, 290),
 (290, 291),
 (291, 295),
 (295, 302),
 (302, 303),
 (303, 307),
 (307, 315),
 (315, 316),
 (316, 320),
 (320, 327),
 (327, 328),
 (328, 332),
 (3

In [80]:
tokenizer._convert_id_to_token(3810)

'▁Pre'

In [8]:
|""" Experiment for ELECTRA get discriminator input
1) flatten logit tensor and label tensor
2) get highest logit
3) masked select for mlm masking index
4) get index of mlm masking index
5) index select for discriminator input 
"""
flat_logit = torch.tensor([99, 98, 97, 96])
test = torch.tensor([-100, -100, 1, 2, 3, 4])
mlm_mask_idx = torch.where(test != -100)

test2 = test.clone()
test[mlm_mask_idx] = flat_logit
test


tensor([-100, -100,   99,   98,   97,   96])

In [10]:
torch.eq(test, test2).long()

tensor([1, 1, 0, 0, 0, 0])

In [18]:

a = torch.tensor([1, 2])

test = torch.tensor([-100, -100, 1, 2, 3, 4])
test.view(-1, a.size(0))

tensor([[-100, -100],
        [   1,    2],
        [   3,    4]])

In [34]:
""" Experiment for Huggingface Tokenizer for Building MLM Algorithm 
meaning of special token in Huggingface Tokenizer is corrspoding to [CLS], [SEP], [MASK], [PAD] ... etc
"""

text = 'I am a boy [MASK] [MASK] are a girl [PAD]'
tokens = CFG.tokenizer(text)
input_ids = [torch.tensor(x) for x in tokens["input_ids"]]
special_tokens_mask = CFG.tokenizer.get_special_tokens_mask(input_ids, already_has_special_tokens=True)
special_tokens_mask

[1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1]

In [35]:
""" Experiment for Huggingface Tokenizer for Building MLM Algorithm 
"""
mlm_probability = 0.15
probability_matrix = torch.full(torch.tensor(input_ids).shape, mlm_probability)

tensor([0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500,
        0.1500, 0.1500, 0.1500])

In [36]:
""" Experiment for Huggingface Tokenizer for Building MLM Algorithm 
"""
probability_matrix.masked_fill_(special_tokens_mask, value=0.0)
masked_indices = torch.bernoulli(probability_matrix).bool()
masked_indices

TypeError: masked_fill_() received an invalid combination of arguments - got (list, value=float), but expected one of:
 * (Tensor mask, Tensor value)
      didn't match because some of the arguments have invalid types: (!list of [int, int, int, int, int, int, int, int, int, int, int, int]!, !value=float!)
 * (Tensor mask, Number value)
      didn't match because some of the arguments have invalid types: (!list of [int, int, int, int, int, int, int, int, int, int, int, int]!, !value=float!)


In [32]:
input_ids[~masked_indices] = -100

TypeError: only integer tensors of a single element can be converted to an index

In [29]:
def hf_load_dataset(cfg: CFG) -> DatasetDict:
    """ Load dataset from Huggingface Datasets
    Notes:
        This function is temporary just fit-able for Wikipedia dataset
    References:
        https://github.com/huggingface/datasets/blob/main/src/datasets/load.py#2247
    """
    dataset = load_dataset(cfg.hf_dataset, cfg.language)
    return dataset


def hf_split_dataset(cfg: CFG, dataset: Dataset) -> Tuple[Dataset, Dataset]:
    """ Split dataset from Huggingface Datasets with huggingface method "train_test_split"
    Args:
        cfg: configuration.CFG, needed to load split ratio, seed value
        dataset: Huggingface Datasets object, dataset from Huggingface Datasets
    Notes:
        This function is temporary just fit-able for Wikipedia dataset & MLM Task
    """
    dataset = dataset.train_test_split(cfg.split_ratio, seed=cfg.seed)
    train, valid = dataset['train'], dataset['test']
    return train, valid


def chunking(sequences: Dict, cfg: CFG = CFG) -> List[str]:
    """ Chunking sentence to token using pretrained tokenizer
    Args:
        cfg: configuration.CFG, needed to load pretrained tokenizer
        sequences: list, sentence to chunking
    References:
        https://huggingface.co/docs/transformers/main/tasks/masked_language_modeling
    """
    return cfg.tokenizer([" ".join(x) for x in sequences['text']])


def group_texts(sequences: Dict, cfg: CFG = CFG) -> Dict:
    """ Dealing Problem: some of data instances are longer than the maximum input length for the model,
    This function is ONLY used to HF Dataset Object
    1) Concatenate all texts
    2) We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    3) customize this part to your needs
    4) Split by chunks of max_len
    """
    concatenated_sequences = {k: sum(sequences[k], []) for k in sequences.keys()}
    total_length = len(concatenated_sequences[list(sequences.keys())[0]])
    if total_length >= cfg.max_seq:
        total_length = (total_length // cfg.max_seq) * cfg.max_seq
    result = {
        k: [t[i: i + cfg.max_seq] for i in range(0, total_length, cfg.max_seq)]
        for k, t in concatenated_sequences.items()
    }
    return result


def apply_preprocess(dataset: Dataset, function: Callable, batched: bool = True, num_proc: int = 4, remove_columns: any = None) -> Dataset:
    """ Apply preprocessing to text data, which is using huggingface dataset method "map()"
    for pretrained training (MLM, CLM)
    Args:
        dataset: Huggingface Datasets object, dataset from Huggingface Datasets
        function: Callable, function that you want to apply
        batched: bool, default True, if you want to apply function to batched data, set True
        num_proc: int, default 4, number of process for multiprocessing
        remove_columns: any, default None, if you want to remove some columns, set column name
    References:
        https://huggingface.co/docs/transformers/main/tasks/masked_language_modeling
    """
    mapped_dataset = dataset.map(
        function,
        batched=batched,
        num_proc=num_proc,
        remove_columns=remove_columns,
    )
    return mapped_dataset


def load_data(data_path: str) -> pd.DataFrame:
    """ Load data_folder from csv file like as train.csv, test.csv, val.csv
    """
    df = pd.read_csv(data_path)
    return df


def no_char(text):
    text = re.sub(r"\s+[a-zA-Z]\s+", " ", text)
    text = re.sub(r"\^[a-zA-Z]\s+", " ", text)
    text = re.sub(r"\s+[a-zA-Z]$", " ", text)
    return text


def no_multi_spaces(text):
    return re.sub(r"\s+", " ", text, flags=re.I)


def underscore_to_space(text: str):
    text = text.replace("_", " ")
    text = text.replace("-", " ")
    return text


def preprocess_text(source):
    """ Remove all the special characters
    """
    source = re.sub(r'\W', ' ', str(source))
    source = re.sub(r'^b\s+', '', source)
    source = source.lower()
    return source


def cleaning_words(text: str) -> str:
    """ Apply all of cleaning process to text data
    """
    tmp_text = underscore_to_space(text)
    tmp_text = no_char(tmp_text)
    tmp_text = preprocess_text(tmp_text)
    tmp_text = no_multi_spaces(tmp_text)
    return tmp_text


def split_token(inputs: str):
    """ Convert malform list to Python List Object & elementwise type casting
    """
    inputs = cleaning_words(inputs)
    tmp = inputs.split()
    result = list(map(int, tmp))
    return result


def split_list(inputs: List, max_length: int) -> List[List]:
    """ Split List into sub shorter list, which is longer than max_length
    """
    result = [inputs[i:i + max_length] for i in range(0, len(inputs), max_length)]
    return result


def flatten_sublist(inputs: List[List], max_length: int = 512) -> List[List]:
    """ Flatten Nested List to 1D-List """
    result = []
    for instance in tqdm(inputs):
        tmp = split_token(instance)
        if len(tmp) > max_length:
            tmp = split_list(tmp, max_length)
            for i in range(len(tmp)):
                result.append(tmp[i])
        else:
            result.append(tmp)
    return result


def preprocess4tokenizer(input_ids: List, token_type_ids: List, attention_mask: List):
    for i, inputs in tqdm(enumerate(input_ids)):
        if inputs[0] != 1:
            inputs.insert(0, 1)
            token_type_ids[i].insert(0, 0)
            attention_mask[i].insert(0, 1)
        if inputs[-1] != 2:
            inputs.append(2)
            token_type_ids[i].append(0)
            attention_mask[i].append(1)
    return input_ids, token_type_ids, attention_mask


def cut_instance(input_ids: List, token_type_ids: List, attention_mask: List, min_length: int = 256):
    n_input_ids, n_token_type_ids, n_attention_mask = [], [], []
    for i, inputs in tqdm(enumerate(input_ids)):
        if len(inputs) >= min_length:
            n_input_ids.append(inputs)
            n_token_type_ids.append(token_type_ids[i])
            n_attention_mask.append(attention_mask[i])
    return n_input_ids, n_token_type_ids, n_attention_mask


def save_pkl(input_dict: Any, filename: str) -> None:
    with open(f'{filename}.pkl', 'wb') as file:
        pickle.dump(input_dict, file)


def load_pkl(filepath: str) -> Any:
    """  Load pickle file
    Examples:
        filepath = './dataset_class/data_folder/train'
    """
    with open(f'{filepath}.pkl', 'rb') as file:
        output = pickle.load(file)
    return output


In [2]:
"""
1) Load Dataset, Tokenizer
2) Split Dataset, preprocess dataset for MLM Task
"""
ds = hf_load_dataset(CFG)
_, sub_ds = hf_split_dataset(CFG, ds['train'])
train, valid = hf_split_dataset(CFG, sub_ds)

NameError: name 'hf_load_dataset' is not defined

In [4]:
""" Apply preprocessing to dataset """

chunked_train = apply_preprocess(
    train,
    chunking,
    remove_columns=train.column_names
)

chunked_valid = apply_preprocess(
    valid,
    chunking,
    remove_columns=valid.column_names
)

Map (num_proc=4):   0%|          | 0/1025250 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/256313 [00:00<?, ? examples/s]

In [8]:
""" Grouping text data to fit the maximum input length for the model """

grouped_train = apply_preprocess(
    chunked_train,
    group_texts,
    num_proc=8,
)

grouped_valid = apply_preprocess(
    chunked_train,
    group_texts,
    num_proc=8,
)

Map (num_proc=8):   0%|          | 0/1025250 [00:00<?, ? examples/s]

TimeoutError: 